# Videos Information2 [update, description]

(1) 실시간 스트리밍경우 소리만 나오는듯 하다(=아닌 경우도 존재하는 듯). 날짜로 스트리밍인지 아닌지 구분 가능.  
(2) 진짜 더보기가 없는 경우  
(3) \<a href의 경우 찾음 1238번째  
(4) 언어 바꿔서 크롤링하기 - 완성  
(5) 동영상이 사용자에 의해 삭제된 상황 처리 39번째 혹은 비공개 동영상인 경우. - 완성  
(6) 더보기가 안 들어간 것도 아니고 이상하게 한두글자 들어간 경우 - 완성  

In [3]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from slacker import Slacker
import time
from datetime import datetime 

## Using slack Bot

In [9]:
f = open("../../slack_token_check_bot.txt")
token_check = f.readline()[:-1]
slack_check = Slacker(token_check)
f.close()
token_check[-3:]

'k9z'

In [10]:
f = open("../../slack_token_noti_bot.txt")
token_noti = f.readline()[:-1]
slack_noti = Slacker(token_noti)
f.close()
token_noti[-3:]

'fh8'

## Read / Make CSV file

In [11]:
csv_name = "youtuber_videos_info2.csv"

In [27]:
try: 
    videos_df = pd.read_csv(csv_name, encoding = "utf-8")
    print("Read file")
except:
    print("Exception handling")
    temp_df = pd.read_csv("youtuber_videos_info.csv", encoding = "utf-8")
    videos_df = temp_df
    videos_df["update"] = None
    videos_df["description"] = None
    videos_df["thumbnail"] = None
    videos_df.to_csv(csv_name, encoding = "utf-8", index = False)
    videos_df = pd.read_csv(csv_name, encoding = "utf-8")
    videos_df = videos_df.fillna(0)
    videos_df.to_csv(csv_name, encoding = "utf-8", index = False)
    videos_df = pd.read_csv(csv_name, encoding = "utf-8")

Exception handling


In [28]:
type(videos_df["update"][4079])

numpy.float64

In [29]:
videos_df.head(3)

,youtuber_name,video_id,update,description,thumbnail
0,씬님,/watch?v=UTXuSv8e18E,0.0,0.0,0.0
1,씬님,/watch?v=_BsClDZgRaY,0.0,0.0,0.0
2,씬님,/watch?v=u_aOZxEiIFs,0.0,0.0,0.0


### Edit video_id

In [30]:
youtube = "https://www.youtube.com"
thumb_pre = "https://i.ytimg.com/vi/"
thumb_des = "/maxresdefault.jpg"
cnt_videos = len(videos_df)

In [32]:
start_time = datetime.now()
cnt_change_df = 0
for i in range(cnt_videos):
    if videos_df["video_id"][i][:19] != "https://www.youtube":
        cnt_change_df += 1
        temp_ad = youtube + videos_df["video_id"][i]
        videos_df["video_id"][i] = temp_ad
        videos_df.to_csv(csv_name, encoding = "utf-8", index = False)
slack_noti.chat.post_message("#result", "*Videos_information* End of change address, Change of number : " + str(cnt_change_df), as_user=True)
print(datetime.now()-start_time)
print("Change of number : ", cnt_change_df)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


0:01:10.956563
Change of number :  2400


# Function

In [56]:
def get_address_bs(ad):
    open_address = driver.get(ad)
    get_source = driver.page_source
    raise
    global address_bs
    address_bs = BeautifulSoup(get_source, "html.parser")
    return address_bs

In [45]:
def private_delete_video(num, ad): #동영상을 볼 수 없습니다. #동영상이 사용자에 의해 삭제되었습니다.
    pride_bs = get_address_bs(ad)  #Content Warning      #removed by the user
    result = str(pride_bs.find_all("h1", {"class" : "message"}))
    while(1):
        if "\n" not in result:
            if "  " not in result:
                break
        result = result.replace("\n", "")
        result = result.replace("  ", "")
    result = result[46:-6]
    if "삭제되었습니다." in result:
        videos_df["update"][num] = "remove"
        videos_df["description"][num] = "remove"
        videos_df["thumbnail"][num] = "remove"
        slack_check.chat.post_message("#output", " *This video removd by user.*\n```Number : " + str(i) + ", "+ videos_df["video_id"] + "```", as_user=True)
    elif "볼 수 없습니다." in result:
        videos_df["update"][num] = "private"
        videos_df["description"][num] = "private"
        videos_df["thumbnail"][num] = "private"
        slack_check.chat.post_message("#output", " *Content Warning*\n```Number : " + str(i) + ", "+ videos_df["video_id"] + "```", as_user=True)

In [42]:
def find_upload_date(ad_bs):
    upload = ad_bs.find_all("div", {"id" : "watch-uploader-info"})
    upload = str(upload)
    upload = upload[63:-16]
    return upload

In [43]:
def find_description(ad_bs):
    global description
    describe = ad_bs.find_all("p", {"id" : "eow-description"})
    describe = str(describe)
    describe = describe[34:-5]
    describe = describe.replace("</a>", "")
    describe = describe.replace("<br/>", "\n")
    describe = describe.split("<a class=\"")
    for i in range(len(describe)):
        if "yt-uix-servicelink" in describe[i]:
            result = describe[i]
            result = result.split("target=\"_blank\">")
            #result = result.split("\">")
            describe[i] = result[1]
        if "yt-uix-sessionlink" in describe[i]:
            result = describe[i]
            result = result.split("\">")
            describe[i] = result[1]
    description = "".join(describe)
    return description

In [88]:
def set_korean():
    open_address = driver.get("https://www.youtube.com/watch?v=TGstCQ5qUeE")
    time.sleep(10)
    get_source = driver.page_source
    set_bs = BeautifulSoup(get_source, "html.parser")
    driver.find_element_by_id('yt-picker-language-button').click()
    time.sleep(5)
    driver.find_element_by_xpath("//*[@title='Korean']").click()

In [89]:
set_korean()

In [92]:
def set_korean2():
    open_address = driver.get("https://www.youtube.com/watch?v=UTXuSv8e18E")
    time.sleep(10)
    get_source = driver.page_source
    #set_bs = BeautifulSoup(get_source, "html.parser")
    #driver.find_element_by_id('yt-picker-language-button').click()
    #time.sleep(5)
    #driver.find_element_by_xpath("//*[@title='Korean']").click()

In [93]:
set_korean2()

# Main

확인할 것  
(1) 비공개/삭제 동영상인 경우  
(2) Description 10자 미만 확인

In [57]:
slack_noti.chat.post_message("#result", "\n" + "> For perfection of data\n *Start* 💣" + str(datetime.now()), as_user=True)
start_time = datetime.now()

short_ds = 0
find_info = 0
driver = webdriver.Firefox()

try:
    for i in range(cnt_videos):
        slack_check.chat.post_message("#process", "*Number : " + str(i) + "*", as_user = True)
        address = videos_df["video_id"][i]
        private_delete_video(i, address)
        if videos_df["update"][i] == 0:
            find_info += 1
            print(i)
            address_bs = get_address_bs(address)
            upload = find_upload_date(address_bs)
            description = find_description(address_bs)
            if len(description) < 10:
                short_ds += 1
                slack_check.chat.post_message("#output", " *Description of less than 10 characters*\n```Number : " + str(i) + ", "+ videos_df["video_id"] + "\n" + description + "```", as_user=True)
            videos_df["update"][i] = upload
            videos_df["description"][i] = description
            video_id = videos_df["video_id"][i][32:]
            thumbnail = thumbnail_pre + video_id + thumbnail_des
            videos_df.to_csv(csv_name, encoding="utf-8", index=False)
except:
    slack_noti.chat.post_message("#result", "*Number : " + str(i) + "* ⚠️ Error ⚠️", as_user=True)
        
driver.quit()
slack_noti.chat.post_message("#result", "```[Summary for perfection]\nFind Information : " + str(find_info) + "\nShort Description : " + str(short_ds) + "\nTime : " + str(datetime.now() - start_time) + "```", as_user=True)
slack_noti.chat.post_message("#result", "*Finish* 🎉" + str(datetime.now()))